In [1]:
import numpy as np
import numpy.matlib
from numpy import exp, abs, sqrt
from scipy.stats import norm
import tensorflow as tf
tf.keras.backend.set_floatx('float64');

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [2]:
def create_features_linspace(N, v_min, v_max):
    '''
    This function creates the features range for the input of the neural network
    
    v_min (array): vector containing the lower bound for each parameter
    v_max (array): vector containing the upper bound for each parameter
    (v_min and v_max have the same lenght)
    '''
    
    nf = len(v_min)
    range_linspace = np.linspace(start = v_min, stop = v_max, num = N)
    features_range = np.zeros((N,nf))  
    for i in range(nf):
        np.random.shuffle(range_linspace[:,i])
        features_range[:,i] = range_linspace[:,i]

    return features_range
    
# test
#create_features_linspace(N = 10, v_min = [0.2, 0.5, 3], v_max = [0.5, 0.9, 7])

In [3]:
class nelson_siegel_curve(object):
    '''
    This class implements the Nelson-Siegel curve function
    
    NS_param (array, 4-dim) : vector of parameters for the Nelson-Siegel curve 
    
    x (float) : evaluation point
    '''
    
    def __init__(self, NS_param):
        self.NS_param = NS_param
        
    def __call__(self, x):
        alpha0, alpha1, alpha2, alpha3 = self.NS_param
        return alpha0 + alpha1*exp(-alpha3*x) + alpha2*alpha3*x*exp(-alpha3*x)
    
    def avg_integral_curve(self, lb, ub):
        '''
        This function calculates the averaged integral of the Nelson Siegel curve.
        lb (float) : lower bound integration
        ub (float) : upper bound integration
        '''
        alpha0, alpha1, alpha2, alpha3 = self.NS_param
        integral_curve = alpha0*(ub-lb)+(alpha1+alpha2+alpha2*alpha3*lb)*exp(-alpha3*lb)/alpha3-(alpha1+alpha2+alpha2*alpha3*ub)*exp(-alpha3*ub)/alpha3
        return integral_curve/(ub-lb)

# test
#f = nelson_siegel_curve(NS_param = [1,-1,1,2])
#f(x = 3)
#f.avg_integral_curve(1/6, 1/12+1/6)

In [4]:
class uni_volatility(object):
    '''
    This class implements the volality function \csi defined in Prop. 3.7.

    model_features (array, 3-dim) : [a, b, k]
        a (float, positive) : volatility coefficient
        b (float, positive) : Samuelson effect coefficient
        k (float, positive) : covariance kernel coefficient
    
    tau (float, positive) : time to delivery
    T_1 (float, positive) : start of delivery
    l (float, positive) : length of the contract
    t (float, positive) : evaluation date (default = 0)
    '''
    
    def __init__(self, model_features):
        self.model_features = model_features
        
    def __call__(self, tau, T_1, l, t = 0):    
        a, b, k = self.model_features
        
        vol = (b**2+3)/3+((3*(2-l)*(l**2)-4)*(b**2)/6+3*l-2)*exp(-b*l) + (b**2+3)*exp(-2*b*l)/3
        vol = vol * (exp(-2*b*(T_1-tau)-exp(-2*b*(T_1-t)))) 
        vol = vol * 2*a**2/((l**2)*(b**5)*k) 
        return sqrt(vol) 
            
# test 
#s = uni_volatility_adj2(model_features = [0.2, 0.8, 7])
#s(tau = 1/12, T_1 = 1/12, l = 1/12)

In [5]:
class black_scholes_price(nelson_siegel_curve):
    '''
    This class implements the function price of an european CALL option, accordling to Proposition 3.11
    NS_param (array, 4-dim) : vector of parameters for the Nelson-Siegel curve 
    
    option_features (array, 4-dim) : [K, tau, T_1, l]
        K (float, positive) : strike price
        tau (float, positive) : time to delivery
        T_1 (float, positive) : start of delivery
        l (float, positive) : length of the contract

    model_features (array, 3-dim) : [a, b, k]
        a (float, positive) : volatility coefficient
        b (float, positive) : Samuelson effect coefficient
        k (float, positive) : covariance kernel coefficient
    
    t (float, positive) : evaluation date (default = 0)
    '''
    
    def __init__(self, NS_param):
        nelson_siegel_curve.__init__(self, NS_param)
        
    def __call__(self, option_features, model_features, t = 0):
        K, tau, T_1, l = option_features
        NS_curve = nelson_siegel_curve(NS_param = self.NS_param)
        s = uni_volatility(model_features = model_features)
        rv = norm()
        
        csi = s(tau = tau, T_1 = T_1, l = l)
        d = (NS_curve.avg_integral_curve(lb = T_1-t, ub = T_1+l-t)-K)/csi
        return csi*rv.pdf(d) + (NS_curve.avg_integral_curve(lb = T_1-t, ub = T_1+l-t)-K)*rv.cdf(d)

# test
#price = black_scholes_price(NS_param = [33, -5, 5, 2])
#price(option_features = [31, 0.1, 0.1, 0.5], model_features = [0.5, 0.636, 7])